# Getter and Setter


In [1]:
from datetime import datetime


class TimeUTC:
    def __get__(self, instance, owner):
        print(f"__get__ called , self ={self} , instance ={instance} ,owner = {owner}")
        return datetime.now().isoformat()

In [9]:
TimeUTC()

In [2]:
class Logger1:
    current_time = TimeUTC()


class Logger2:
    current_time = TimeUTC()

In [3]:
Logger1.current_time

__get__ called , self =<__main__.TimeUTC object at 0x000002121DC69E90> , instance =None ,owner = <class '__main__.Logger1'>


'2022-11-25T13:07:05.438796'

As you can see, the instance was None - this was because we called the descriptor from the Logger1 class, not an instance of it. The owner_class tells us this descriptor instance is defined in the Logger1 class.

In [4]:
Logger2.current_time

__get__ called , self =<__main__.TimeUTC object at 0x000002121DC91050> , instance =None ,owner = <class '__main__.Logger2'>


'2022-11-25T13:08:20.009084'

In [5]:
l1 = Logger1()
l2 = Logger1()

In [8]:
hex(id(l1)), hex(id(l2))

('0x2121d566590', '0x2121d567a50')

In [7]:
l1.current_time, l2.current_time

__get__ called , self =<__main__.TimeUTC object at 0x000002121DC69E90> , instance =<__main__.Logger1 object at 0x000002121D566590> ,owner = <class '__main__.Logger1'>
__get__ called , self =<__main__.TimeUTC object at 0x000002121DC69E90> , instance =<__main__.Logger1 object at 0x000002121D567A50> ,owner = <class '__main__.Logger1'>


('2022-11-25T13:08:49.050230', '2022-11-25T13:08:49.050230')

This means that we can differentiate, inside our __get__ method whether the descriptor was accessed via the class or via an instance.

Typically when a descriptor is access from the class we return the descriptor instance, and when accessed from the instance we return the instance specific value we want

In [10]:
class TimeUTC:
    def __get__(self, instance, owner):
        if instance is None:
            return self
        return datetime.now().isoformat()

In [17]:
TimeUTC()

In [11]:
class Logger:
    current_time = TimeUTC()

In [12]:
Logger.current_time

In [13]:
#? calling from the instance
Logger().current_time

'2022-11-25T13:12:40.079464'

This is consistent with the way properties work:

In [14]:
class Logger:
    @property
    def current_time(self):
        return datetime.now().isoformat()

In [15]:
Logger.current_time

In [16]:
Logger().current_time

'2022-11-25T13:13:57.595777'

Now, there is one subtle point we have to understand when we create multiple instances of a class that uses a descriptor as a class attribute.

Since the descriptor is assigned to an class attribute, all instances of the class will share the same descriptor instance!

In [18]:
class TimeUTC:
    def __get__(self, instance, owner_class):
        if instance is None:
            # called from class
            return self
        else:
            # called from instance
            print(f'__get__ called in {self}')
            return datetime.utcnow().isoformat()


class Logger:
    current_time = TimeUTC()

In [19]:
l1 = Logger()
l2 = Logger()

In [20]:
l1.current_time, l2.current_time

__get__ called in <__main__.TimeUTC object at 0x000002121DFA08D0>
__get__ called in <__main__.TimeUTC object at 0x000002121DFA08D0>


('2022-11-25T07:45:34.299945', '2022-11-25T07:45:34.299945')

As you can see the same instance of TimeUTC was used.

This does not matter in this particular example, since we just return the current time, but watch what happens if our property relies on some kind of state in the descriptor:

In [30]:
class CountDown:
    def __init__(self, start):
        self.start = start + 1

    def __get__(self, instance, owner):
        if instance is None:
            return self
        self.start -= 1
        return self.start

In [31]:
class Rocket:
    countdown = CountDown(10)

In [32]:
r1 = Rocket()
r2 = Rocket()

In [33]:
r1.countdown

10

In [34]:
r1.countdown

9

In [35]:
#? starting the countdown for rocket2
r2.countdown

7

As you can see, the current countdown value is shared by both rocket1 and rocket2 instances of Rocket - this is because the Countdown instance is a class attribute of Rocket. So we have to be careful how we deal with instance level state.

The __set__ method works in a similar way to __get__ but it is used when we assign a value to the class attribute

In [42]:
class IntegerValue:
    def __get__(self, instance, owner):
        if instance is None:
            return self
        return self._value
    def __set__(self, instance, value):
        self._value = int(value)

#! storing the value in the descriptor instance, this bad

In [43]:
class Point2D:
    x = IntegerValue()
    y = IntegerValue()

In [44]:
p1 = Point2D()
p2 = Point2D()

In [45]:
p1.x = 100.2
p1.y = 200.5

In [46]:
p1.x,p1.y

(100, 200)

In [47]:
#? i didnt store any value in the p2 instance
p2.x,p2.y

(100, 200)

In [48]:
p2.x = 1000.1

In [50]:
p1.x,p1.y

(1000, 200)

So, obviously using the descriptor instance dictionary for storage at the instance level is probably not going to work in most cases!

And this is the reason both the __get__ and __set__ methods need to know which instance we are dealing with